
# 🤖 Aula 6: Criando um Chatbot que Aprende Sozinho

Neste notebook, vamos construir um chatbot que aprende com as interações dos usuários.

## 📌 O que faremos:
1. Criar um histórico de mensagens para aprendizado contínuo.
2. Implementar um chatbot que ajusta suas respostas com base no feedback do usuário.
3. Testar personalização e adaptação do chatbot.

---


In [ ]:

# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai json


In [ ]:

import openai
import json

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um histórico de interações para aprendizado contínuo
historico = []

def chatbot_aprendiz(pergunta):
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    
    # Salvando histórico em um arquivo JSON
    with open("historico.json", "w") as f:
        json.dump(historico, f, indent=4)
    
    return mensagem

# Teste do chatbot
print(chatbot_aprendiz("Qual é a capital da França?"))
print(chatbot_aprendiz("E qual é a minha cor favorita?"))


In [ ]:

# Adicionando feedback para aprendizado contínuo
def chatbot_com_feedback(pergunta):
    resposta = chatbot_aprendiz(pergunta)
    print("Bot:", resposta)
    feedback = input("Isso ajudou? (Sim/Não) ").strip().lower()
    
    if feedback == "não":
        print("Obrigado! Vou melhorar minha resposta na próxima vez.")
    
    return resposta

# Testando com feedback
print(chatbot_com_feedback("Me fale sobre inteligência artificial."))


In [ ]:

# Criando um chatbot que lembra do usuário
usuarios = {}

def chatbot_personalizado(nome, pergunta):
    if nome not in usuarios:
        usuarios[nome] = []
    
    usuarios[nome].append(pergunta)
    
    resposta = chatbot_aprendiz(pergunta)
    
    return f"{nome}, baseado no que já conversamos, aqui está minha resposta: {resposta}"

# Teste de personalização
print(chatbot_personalizado("Carlos", "Qual celular devo comprar?"))
print(chatbot_personalizado("Carlos", "Qual minha preferência de marca?"))



## 🎯 Desafio para os alunos!
1. Criar um chatbot que lembra informações do usuário ao longo do tempo.
2. Implementar um mecanismo de feedback para melhorar respostas.
3. Testar personalização e adaptação do chatbot em conversas mais longas.

---
